In [15]:
#Importar e instalar
import pandas as pd
import numpy as np
import pylab as plt   
import seaborn as sns
import datetime
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error
from sklearn.preprocessing import OneHotEncoder
import warnings
warnings.filterwarnings('ignore')   # para quitar esos prints

In [16]:
data = pd.read_csv('data/fusion_MG_UNICEF_1.0_all.csv', sep=',', encoding='utf-8')

In [17]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 37699 entries, 0 to 37698
Data columns (total 18 columns):
 #   Column                                                                          Non-Null Count  Dtype  
---  ------                                                                          --------------  -----  
 0   DATAFLOW                                                                        37699 non-null  object 
 1   REF_AREA:Geographic area                                                        37699 non-null  object 
 2   INDICATOR:Indicator                                                             37699 non-null  object 
 3   AGE:Current age                                                                 37699 non-null  object 
 4   STAT_POP:Statistical Population                                                 37699 non-null  object 
 5   TIME_PERIOD:Time period                                                         37699 non-null  int64  
 6   OBS_VALUE:Obse

In [18]:
# Eliminar las columnas con valores nulos
df = data.dropna(axis=1)

In [19]:
# Verificar los resultados
df.head()

,DATAFLOW,REF_AREA:Geographic area,INDICATOR:Indicator,AGE:Current age,STAT_POP:Statistical Population,TIME_PERIOD:Time period,OBS_VALUE:Observation Value,UNIT_MEASURE:Unit of measure,SOURCE_LINK:Citation of or link to the data source,DATA_SOURCE:Data Source,OBS_FOOTNOTE:Observation footnote
0,UNICEF:MG(1.0): Migration,BDI: Burundi,MG_INTNL_MG_CNTRY_DEST: International migrants...,_T: Total,_T: Total,1990,333,PS: Persons,https://www.un.org/development/desa/pd/content...,United Nations Department of Economic and Soci...,235 countries/areas where migration data avail...
1,UNICEF:MG(1.0): Migration,BDI: Burundi,MG_INTNL_MG_CNTRY_DEST: International migrants...,_T: Total,_T: Total,1995,255,PS: Persons,https://www.un.org/development/desa/pd/content...,United Nations Department of Economic and Soci...,235 countries/areas where migration data avail...
2,UNICEF:MG(1.0): Migration,BDI: Burundi,MG_INTNL_MG_CNTRY_DEST: International migrants...,_T: Total,_T: Total,2000,126,PS: Persons,https://www.un.org/development/desa/pd/content...,United Nations Department of Economic and Soci...,235 countries/areas where migration data avail...
3,UNICEF:MG(1.0): Migration,BDI: Burundi,MG_INTNL_MG_CNTRY_DEST: International migrants...,_T: Total,_T: Total,2005,193,PS: Persons,https://www.un.org/development/desa/pd/content...,United Nations Department of Economic and Soci...,235 countries/areas where migration data avail...
4,UNICEF:MG(1.0): Migration,BDI: Burundi,MG_INTNL_MG_CNTRY_DEST: International migrants...,_T: Total,_T: Total,2010,247,PS: Persons,https://www.un.org/development/desa/pd/content...,United Nations Department of Economic and Soci...,235 countries/areas where migration data avail...


In [20]:
df.tail()

,DATAFLOW,REF_AREA:Geographic area,INDICATOR:Indicator,AGE:Current age,STAT_POP:Statistical Population,TIME_PERIOD:Time period,OBS_VALUE:Observation Value,UNIT_MEASURE:Unit of measure,SOURCE_LINK:Citation of or link to the data source,DATA_SOURCE:Data Source,OBS_FOOTNOTE:Observation footnote
37694,UNICEF:MG(1.0): Migration,UNICEF_MENARO: Middle East and North Africa,MG_UNRWA_RFGS_CNTRY_ASYLM: Refugees under UNRW...,Y0T17: Under 18 years old,_T: Total,2020,1770322.0,PS: Persons,https://www.unrwa.org/what-we-do/relief-and-so...,The United Nations Relief and Works Agency for...,From UNRWA: UNRWA registered population dashbo...
37695,UNICEF:MG(1.0): Migration,UNICEF_MENARO: Middle East and North Africa,MG_UNRWA_RFGS_CNTRY_ASYLM: Refugees under UNRW...,Y0T17: Under 18 years old,_T: Total,2021,1769762.0,PS: Persons,https://www.unrwa.org/what-we-do/relief-and-so...,The United Nations Relief and Works Agency for...,From UNRWA: UNRWA registered population dashbo...
37696,UNICEF:MG(1.0): Migration,WORLD: World,MG_UNRWA_RFGS_CNTRY_ASYLM: Refugees under UNRW...,Y0T17: Under 18 years old,_T: Total,2019,1819934.0,PS: Persons,https://www.unrwa.org/what-we-do/relief-and-so...,The United Nations Relief and Works Agency for...,From UNRWA: UNRWA registered population dashbo...
37697,UNICEF:MG(1.0): Migration,WORLD: World,MG_UNRWA_RFGS_CNTRY_ASYLM: Refugees under UNRW...,Y0T17: Under 18 years old,_T: Total,2020,1770322.0,PS: Persons,https://www.unrwa.org/what-we-do/relief-and-so...,The United Nations Relief and Works Agency for...,From UNRWA: UNRWA registered population dashbo...
37698,UNICEF:MG(1.0): Migration,WORLD: World,MG_UNRWA_RFGS_CNTRY_ASYLM: Refugees under UNRW...,Y0T17: Under 18 years old,_T: Total,2021,1769762.0,PS: Persons,https://www.unrwa.org/what-we-do/relief-and-so...,The United Nations Relief and Works Agency for...,From UNRWA: UNRWA registered population dashbo...


In [21]:
# Contar la frecuencia de cada valor en la columna 'UNIT_MEASURE:Unit of measure'
unit_measure_counts = df['UNIT_MEASURE:Unit of measure'].value_counts()
print(unit_measure_counts)

PS: Persons                                  16971
NUMBER: Number                               12702
PCNT: %                                       7614
PER1000: Per 1000                              207
PER1USD_GNI_CAP: per 1 USD GNI per capita      205
Name: UNIT_MEASURE:Unit of measure, dtype: int64


In [22]:
# Filtrar el DataFrame original manteniendo solo las filas con 'UNIT_MEASURE:Unit of measure' igual a 'PS: Persons'
df = df[df['UNIT_MEASURE:Unit of measure'] == 'PS: Persons']

# Mostrar el DataFrame actualizado
df.head()

,DATAFLOW,REF_AREA:Geographic area,INDICATOR:Indicator,AGE:Current age,STAT_POP:Statistical Population,TIME_PERIOD:Time period,OBS_VALUE:Observation Value,UNIT_MEASURE:Unit of measure,SOURCE_LINK:Citation of or link to the data source,DATA_SOURCE:Data Source,OBS_FOOTNOTE:Observation footnote
0,UNICEF:MG(1.0): Migration,BDI: Burundi,MG_INTNL_MG_CNTRY_DEST: International migrants...,_T: Total,_T: Total,1990,333,PS: Persons,https://www.un.org/development/desa/pd/content...,United Nations Department of Economic and Soci...,235 countries/areas where migration data avail...
1,UNICEF:MG(1.0): Migration,BDI: Burundi,MG_INTNL_MG_CNTRY_DEST: International migrants...,_T: Total,_T: Total,1995,255,PS: Persons,https://www.un.org/development/desa/pd/content...,United Nations Department of Economic and Soci...,235 countries/areas where migration data avail...
2,UNICEF:MG(1.0): Migration,BDI: Burundi,MG_INTNL_MG_CNTRY_DEST: International migrants...,_T: Total,_T: Total,2000,126,PS: Persons,https://www.un.org/development/desa/pd/content...,United Nations Department of Economic and Soci...,235 countries/areas where migration data avail...
3,UNICEF:MG(1.0): Migration,BDI: Burundi,MG_INTNL_MG_CNTRY_DEST: International migrants...,_T: Total,_T: Total,2005,193,PS: Persons,https://www.un.org/development/desa/pd/content...,United Nations Department of Economic and Soci...,235 countries/areas where migration data avail...
4,UNICEF:MG(1.0): Migration,BDI: Burundi,MG_INTNL_MG_CNTRY_DEST: International migrants...,_T: Total,_T: Total,2010,247,PS: Persons,https://www.un.org/development/desa/pd/content...,United Nations Department of Economic and Soci...,235 countries/areas where migration data avail...


In [23]:
# Utilizar pivot_table para convertir los años en columnas y sumar los valores
pivot_df = pd.pivot_table(df, values='OBS_VALUE:Observation Value', index=['REF_AREA:Geographic area', 'INDICATOR:Indicator', 'AGE:Current age'], columns='TIME_PERIOD:Time period', aggfunc=sum)
pivot_df.head()

TIME_PERIOD:Time period                                                                               1990  \
REF_AREA:Geographic area INDICATOR:Indicator                                AGE:Current age                  
ABW: Aruba               MG_INTNL_MG_CNTRY_DEST: International migrants,... Y0T17: Under 18 years old    3   
                                                                            _T: Total                   14   
                         MG_RFGS_CNTRY_ASYLM: Refugees, by country of as... _T: Total                  NaN   
                         MG_RFGS_CNTRY_ORIGIN: Refugees, by country of o... _T: Total                  NaN   
AFG: Afghanistan         MG_INTERNAL_DISP_PERS: Internally displaced per... Y0T17: Under 18 years old  NaN   

TIME_PERIOD:Time period                                                                               1995  \
REF_AREA:Geographic area INDICATOR:Indicator                                AGE:Current age                  
ABW: Aruba               MG_INTNL_MG_CNTRY_DEST: International migrants,... Y0T17: Under 18 years old    4   
                                                                            _T: Total                   22   
                         MG_RFGS_CNTRY_ASYLM: Refugees, by country of as... _T: Total                  NaN   
                         MG_RFGS_CNTRY_ORIGIN: Refugees, by country of o... _T: Total                  NaN   
AFG: Afghanistan         MG_INTERNAL_DISP_PERS: Internally displaced per... Y0T17: Under 18 years old  NaN   

TIME_PERIOD:Time period                                                                               2000  \
REF_AREA:Geographic area INDICATOR:Indicator                                AGE:Current age                  
ABW: Aruba               MG_INTNL_MG_CNTRY_DEST: International migrants,... Y0T17: Under 18 years old    5   
                                                                            _T: Total                   30   
                         MG_RFGS_CNTRY_ASYLM: Refugees, by country of as... _T: Total                  NaN   
                         MG_RFGS_CNTRY_ORIGIN: Refugees, by country of o... _T: Total                  NaN   
AFG: Afghanistan         MG_INTERNAL_DISP_PERS: Internally displaced per... Y0T17: Under 18 years old  NaN   

TIME_PERIOD:Time period                                                                               2001  \
REF_AREA:Geographic area INDICATOR:Indicator                                AGE:Current age                  
ABW: Aruba               MG_INTNL_MG_CNTRY_DEST: International migrants,... Y0T17: Under 18 years old  NaN   
                                                                            _T: Total                  NaN   
                         MG_RFGS_CNTRY_ASYLM: Refugees, by country of as... _T: Total                  NaN   
                         MG_RFGS_CNTRY_ORIGIN: Refugees, by country of o... _T: Total                  NaN   
AFG: Afghanistan         MG_INTERNAL_DISP_PERS: Internally displaced per... Y0T17: Under 18 years old  NaN   

TIME_PERIOD:Time period                                                                               2002  \
REF_AREA:Geographic area INDICATOR:Indicator                                AGE:Current age                  
ABW: Aruba               MG_INTNL_MG_CNTRY_DEST: International migrants,... Y0T17: Under 18 years old  NaN   
                                                                            _T: Total                  NaN   
                         MG_RFGS_CNTRY_ASYLM: Refugees, by country of as... _T: Total                  NaN   
                         MG_RFGS_CNTRY_ORIGIN: Refugees, by country of o... _T: Total                  NaN   
AFG: Afghanistan         MG_INTERNAL_DISP_PERS: Internally displaced per... Y0T17: Under 18 years old  NaN   

TIME_PERIOD:Time period                                                                               2003  \
REF_AREA:Geographic area INDICATOR:

In [24]:
# Eliminar filas con valores NaN
df = df.dropna()

In [25]:
# Seleccionar las columnas relevantes para la predicción
features = ['REF_AREA:Geographic area', 'INDICATOR:Indicator', 'AGE:Current age', 'TIME_PERIOD:Time period']
target = 'OBS_VALUE:Observation Value'
X = df[features]
y = df[target]

In [26]:
# Codificación one-hot de las variables categóricas
encoder = OneHotEncoder(sparse=False)
X_encoded = pd.DataFrame(encoder.fit_transform(X.select_dtypes(include=['object'])))
X_encoded.columns = encoder.get_feature_names(X.select_dtypes(include=['object']).columns)
X_numeric = pd.concat([X.select_dtypes(exclude=['object']), X_encoded], axis=1)

In [27]:
'''
# Crear una copia del DataFrame filtrado
df_prediccion= df.copy()
'''


'\n# Crear una copia del DataFrame filtrado\ndf_prediccion= df.copy()\n'

In [28]:
'''
# Convertir la columna 'TIME_PERIOD:Time period' en tipo datetime
df['TIME_PERIOD:Time period'] = pd.to_datetime(df['TIME_PERIOD:Time period'], format='%Y')


SyntaxError: EOF while scanning triple-quoted string literal (3720330688.py, line 3)

In [ ]:
'''
# Eliminar las filas con valores no numéricos en la columna 'OBS_VALUE:Observation Value'
df_prediccion = df_prediccion.dropna(subset=['OBS_VALUE:Observation Value'])

In [ ]:
'''
# Convertir los valores no numéricos en NaN en la columna 'OBS_VALUE:Observation Value'
df_prediccion['OBS_VALUE:Observation Value'] = pd.to_numeric(df_prediccion['OBS_VALUE:Observation Value'], errors='coerce')

In [ ]:
'''
# Filtrar los datos históricos
df_historico = df[df['TIME_PERIOD:Time period'].dt.year < 2022]

# Filtrar los datos para la predicción
df_prediccion = df[df['TIME_PERIOD:Time period'].dt.year >= 2022]

In [ ]:
'''
# Crear el modelo de regresión lineal
modelo = LinearRegression()

In [ ]:
'''
# Entrenar el modelo utilizando los datos históricos
X = df_historico['OBS_VALUE:Observation Value'].dt.year.values.reshape(-1, 1)
y = df_historico['TIME_PERIOD:Time period'].values.reshape(-1, 1)
modelo.fit(X, y)
'''



"\n# Entrenar el modelo utilizando los datos históricos\nX = df_historico['OBS_VALUE:Observation Value'].dt.year.values.reshape(-1, 1)\ny = df_historico['TIME_PERIOD:Time period'].values.reshape(-1, 1)\nmodelo.fit(X, y)\n"

In [ ]:
'''
# Realizar la predicción para los datos del 2022
X_prediccion = df_prediccion['TIME_PERIOD:Time period'].dt.year.values.reshape(-1, 1)
y_prediccion = modelo.predict(X_prediccion)
'''

"\n# Realizar la predicción para los datos del 2022\nX_prediccion = df_prediccion['TIME_PERIOD:Time period'].dt.year.values.reshape(-1, 1)\ny_prediccion = modelo.predict(X_prediccion)\n"

In [ ]:
'''
if not df_historico.empty:
    # Realizar la predicción para los datos del 2022
    X_prediccion = df_prediccion['TIME_PERIOD:Time period'].dt.year.values.reshape(-1, 1)
    y_prediccion = modelo.predict(X_prediccion)

    # Agregar las predicciones al DataFrame de predicción
    df_prediccion['OBS_VALUE:Observation Value'] = y_prediccion

    # Combinar los datos históricos y las predicciones en un único DataFrame
    df_final = pd.concat([df_historico, df_prediccion])

    # Restaurar el orden original de las filas
    df_final = df_final.sort_index().reset_index(drop=True)

    # Mostrar el DataFrame con las predicciones para 2022
    print(df_final)
else:
    print("No hay suficientes datos históricos para realizar la predicción.")
    '''
    


'\nif not df_historico.empty:\n    # Realizar la predicción para los datos del 2022\n    X_prediccion = df_prediccion[\'TIME_PERIOD:Time period\'].dt.year.values.reshape(-1, 1)\n    y_prediccion = modelo.predict(X_prediccion)\n\n    # Agregar las predicciones al DataFrame de predicción\n    df_prediccion[\'OBS_VALUE:Observation Value\'] = y_prediccion\n\n    # Combinar los datos históricos y las predicciones en un único DataFrame\n    df_final = pd.concat([df_historico, df_prediccion])\n\n    # Restaurar el orden original de las filas\n    df_final = df_final.sort_index().reset_index(drop=True)\n\n    # Mostrar el DataFrame con las predicciones para 2022\n    print(df_final)\nelse:\n    print("No hay suficientes datos históricos para realizar la predicción.")\n    '